# Imports, config, session and utilities

In [1]:
from pathlib import Path
import sys

In [2]:
current_file_path = Path().resolve()
sys.path.insert(0, str(current_file_path.parent))

In [3]:
import pandas as pd
# These import statement after adding parent file path to recognize src python files
from src.config import get_settings
from src.snowflake_client import create_snowpark_session

In [4]:
# Getting .env config and init snowflake session
settings = get_settings()
session = create_snowpark_session(settings)

# Main code

## Programmatic access token (A faire qu'une seule fois)

In [ ]:

query_setup_cortex = """
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';
"""
session.sql(query_setup_cortex).collect()

[Row(status='Statement executed successfully.')]

In [ ]:

user = "sushiatomique"
query = f"""ALTER USER {user.upper()} ADD PROGRAMMATIC ACCESS TOKEN pat;"""
res = session.sql(query).collect()

# Clé api pour le llm snowflake, à mettre dans votre .env
res[0][1]

SnowparkSQLException: (1304): 01c21059-0001-8aaf-0000-0001fb0a52cd: 099401 (22000): Programmatic access token PAT already exists.

In [ ]:
query = f"""
CREATE OR REPLACE NETWORK POLICY cortex_policy
ALLOWED_IP_LIST = (
  '0.0.0.0/0'
);
"""
session.sql(query).collect()


In [24]:

query = f"""
ALTER USER {user} SET NETWORK_POLICY = CORTEX_POLICY
"""

session.sql(query).collect()

[Row(status='Statement executed successfully.')]

## Simple SQL DB interaction

In [10]:
df = pd.DataFrame({
    "id": [0, 1, 2],
    "text": ["text1", "text2", "text3"]
})

In [11]:
warehouse = "SNOWFLAKE_LEARNING_DB"
schema = "PUBLIC"
table_name = "TEST_TABLE"

query = f"""
CREATE TABLE IF NOT EXISTS {warehouse}.{schema}.{table_name} (
    id INT,
    text VARCHAR
);
"""

session.sql(query).collect()

[Row(status='TEST_TABLE already exists, statement succeeded.')]

In [29]:
for index, row in df.iterrows():
    insert_query = f"""
    INSERT INTO {warehouse}.{schema}.{table_name} (id, text)
    VALUES ({row['id']}, '{row['text']}');
    """
    session.sql(insert_query).collect()

In [12]:
query = f"""
SELECT * FROM {warehouse}.{schema}.{table_name}
LIMIT 2;
"""

res = session.sql(query).collect()

In [13]:
res

[Row(ID=0, TEXT='text1'), Row(ID=1, TEXT='text2')]

## OCR and pdf reading

In [5]:
root_folder = Path("/".join(current_file_path.as_posix().split("/")[:-1]))

In [22]:
import pdfplumber

pdf_path = root_folder / "data" / "tableau_garantie.pdf"

# Read and parse PDF
text_content = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text_content += page.extract_text() or ""

print(text_content)


DNIETNASGDT_WZ
Assurén°:24747997-CHENG JEWIN
N°Contrat:2108263;2108267
Dated'effet:04/07/2024
RéférencesCJ:MHPM2200002039S;MHPM2200002040S
RéférencesCJC:M1/O04012-0001-S
24747997 - JEWIN CHENG
BASEOBL-BASEFSS2023 SURCOOBL-SANTE
RG GENERALE
Contratobligatoire Contratobligatoire
Hospitalisation
Fraisdeséjour
Fraisdeséjourconventionnés
Fraisdeséjourconventionnés 600%BR 600%BR-contratdebase
Fraisdeséjournonconventionnés
90%FRlimitéà600%BR,-contratde
Fraisdeséjournonconventionnés 90%FRlimitéà600%BR
base
Honoraires
Honorairesconventionnés
OPTAM/OPTAM-CO:600%BR-
OPTAM/OPTAM-CO:600%BR contratdebase
Honorairesconventionnés
NONOPTAM:TM+100%BR NONOPTAM:200%TM+600%BR
-contratdebase
Honorairesnonconventionnés
90%FRlimitéàTM+100%BR-
Honorairesnonconventionnés 90%FRlimitéàTM+100%BR
contratdebase
Forfaitjournalierhospitalier
Forfaitjournalierhospitalier 20€/jour 20€/jour-contratdebase
Forfaitjournalierpsychiatrie 15€/jour 15€/jour-contratdebase
ForfaitPatientUrgence
ForfaitPatientUrgence 19,61€/jour 1

In [8]:
image_path = root_folder / "data" / "id_card.jpg"  # adjust filename as needed

In [ ]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')
image_path = root_folder / "data" / "id_card.jpg"

result = ocr.ocr(str(image_path))
for line in result[0]:
    print(line[1][0])  # prints the detected text

## LLM usage

In [31]:
from openai import OpenAI

client = OpenAI(api_key=settings.openai_api_key, base_url=settings.openai_api_base)

prompt = "Explain what machine learning is in 2 sentences."

response = client.chat.completions.create(
    model="claude-3-7-sonnet",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

Machine learning is a subset of artificial intelligence where computer systems learn patterns from data and improve their performance on specific tasks without being explicitly programmed. These systems analyze large datasets to identify relationships, make predictions, or take actions, gradually enhancing their accuracy through experience rather than following pre-defined rules.


In [35]:
# Alternative: Use Cortex via SQL instead of the Python SDK
prompt = "Explain what is an insurance."

response = session.sql(f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-7-sonnet', '{prompt}')
""").collect()

print(response[0][0])

# Insurance Explained

Insurance is a financial arrangement where an individual or entity (the policyholder) pays a regular fee (premium) to an insurance company in exchange for a promise of compensation in case of specific losses, damages, or events.

## Key Elements of Insurance:

1. **Risk Transfer**: Insurance transfers the financial risk of certain events from the policyholder to the insurer.

2. **Premiums**: Regular payments made to maintain coverage.

3. **Policy**: A contract detailing what is covered, exclusions, limits, and conditions.

4. **Claims**: Requests for compensation when covered events occur.

5. **Deductibles**: The amount the policyholder must pay before insurance coverage begins.

## Common Types of Insurance:

- Health insurance
- Auto insurance
- Homeowners/renters insurance
- Life insurance
- Disability insurance
- Business insurance

Insurance works on the principle of risk pooling—many people contribute to a fund that pays out to the few who experience los

# Close session 

In [36]:
session.close()